In [2]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [3]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print(use_cuda)

False


In [4]:
pd.set_option('display.max_rows', None)
train_csv = pd.read_csv("train.csv")
test_csv = pd.read_csv("test.csv")
samples_submission_csv = pd.read_csv("sample_submission.csv")

In [9]:
# creation label based on train set
#print(train_csv.dtypes)
train_label1 = train_csv['Id']
print(type(train_label1))
train_label2 = train_csv['SalePrice']
print(type(train_label2))
train_label = pd.DataFrame(columns = ['Id', 'SalePrice'])
train_label['Id'] = train_label1
train_label['SalePrice'] = train_label2
print(train_label)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
        Id  SalePrice
0        1     208500
1        2     181500
2        3     223500
3        4     140000
4        5     250000
5        6     143000
6        7     307000
7        8     200000
8        9     129900
9       10     118000
10      11     129500
11      12     345000
12      13     144000
13      14     279500
14      15     157000
15      16     132000
16      17     149000
17      18      90000
18      19     159000
19      20     139000
20      21     325300
21      22     139400
22      23     230000
23      24     129900
24      25     154000
25      26     256300
26      27     134800
27      28     306000
28      29     207500
29      30      68500
30      31      40000
31      32     149350
32      33     179900
33      34     165500
34      35     277500
35      36     309000
36      37     145000
37      38     153000
38      39     109000
39      40      82000
40      41     160000
41  

In [10]:
# Trainin set
print(train_csv.shape)
train_wo_SP = train_csv.drop('SalePrice', axis='columns')
print(train_wo_SP.shape)



(1460, 81)
(1460, 80)


In [11]:
all_features = pd.concat([train_wo_SP, test_csv], keys=["train", "test"])

all_features_dummies = pd.get_dummies(all_features)

#print(all_features_dummies.shape)
print(all_features_dummies.loc['train'].head)

<bound method NDFrame.head of         Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0        1          60         65.0     8450            7            5   
1        2          20         80.0     9600            6            8   
2        3          60         68.0    11250            7            5   
3        4          70         60.0     9550            7            5   
4        5          60         84.0    14260            8            5   
5        6          50         85.0    14115            5            5   
6        7          20         75.0    10084            8            5   
7        8          60          NaN    10382            7            6   
8        9          50         51.0     6120            7            5   
9       10         190         50.0     7420            5            6   
10      11          20         70.0    11200            5            5   
11      12          60         85.0    11924            9            5   
12      

In [35]:
def split_train_valid_data(data, perc=0.7):
    import pandas as pd
    return data.head(int(len(data)*(perc)))

train_data = split_train_valid_data(all_features_dummies.loc['train'])
valid_data = all_features_dummies.loc['train'].iloc[max(train_data.index+1):]

print(train_data)

        Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0        1          60         65.0     8450            7            5   
1        2          20         80.0     9600            6            8   
2        3          60         68.0    11250            7            5   
3        4          70         60.0     9550            7            5   
4        5          60         84.0    14260            8            5   
5        6          50         85.0    14115            5            5   
6        7          20         75.0    10084            8            5   
7        8          60          NaN    10382            7            6   
8        9          50         51.0     6120            7            5   
9       10         190         50.0     7420            5            6   
10      11          20         70.0    11200            5            5   
11      12          60         85.0    11924            9            5   
12      13          20          NaN   

In [36]:
# same for label

label_train = split_train_valid_data(label)
label_valid = label.iloc[max(train_data.index+1):]

print(label_valid)

        Id  SalePrice
1021  1022     194000
1022  1023      87000
1023  1024     191000
1024  1025     287000
1025  1026     112500
1026  1027     167500
1027  1028     293077
1028  1029     105000
1029  1030     118000
1030  1031     160000
1031  1032     197000
1032  1033     310000
1033  1034     230000
1034  1035     119750
1035  1036      84000
1036  1037     315500
1037  1038     287000
1038  1039      97000
1039  1040      80000
1040  1041     155000
1041  1042     173000
1042  1043     196000
1043  1044     262280
1044  1045     278000
1045  1046     139600
1046  1047     556581
1047  1048     145000
1048  1049     115000
1049  1050      84900
1050  1051     176485
1051  1052     200141
1052  1053     165000
1053  1054     144500
1054  1055     255000
1055  1056     180000
1056  1057     185850
1057  1058     248000
1058  1059     335000
1059  1060     220000
1060  1061     213500
1061  1062      81000
1062  1063      90000
1063  1064     110500
1064  1065     154000
1065  1066

In [37]:
# Recap
print('before')

print(train_data.shape)
print(valid_data.shape)
print(label_train.shape)
print(label_valid.shape)

print('after')
train_data = train_data.drop(['Id'], axis=1)
print(train_data.shape)
valid_data = valid_data.drop(['Id'],axis=1)
print(valid_data.shape)
label_train = label_train.drop(['Id'], axis=1)
print(label_train.shape)
label_valid = label_valid.drop(['Id'], axis=1)
print(label_valid.shape)


before
(1021, 289)
(439, 289)
(1021, 2)
(439, 2)
after
(1021, 288)
(439, 288)
(1021, 1)
(439, 1)


In [38]:
print(label_train)

      SalePrice
0        208500
1        181500
2        223500
3        140000
4        250000
5        143000
6        307000
7        200000
8        129900
9        118000
10       129500
11       345000
12       144000
13       279500
14       157000
15       132000
16       149000
17        90000
18       159000
19       139000
20       325300
21       139400
22       230000
23       129900
24       154000
25       256300
26       134800
27       306000
28       207500
29        68500
30        40000
31       149350
32       179900
33       165500
34       277500
35       309000
36       145000
37       153000
38       109000
39        82000
40       160000
41       170000
42       144000
43       130250
44       141000
45       319900
46       239686
47       249700
48       113000
49       127000
50       177000
51       114500
52       110000
53       385000
54       130000
55       180500
56       172500
57       196500
58       438780
59       124900
60       158000
61      

In [39]:
data_train_label = pd.concat([train_data, label_train], keys=["data", "label"])
print(data_train_label.loc['label']['SalePrice'])

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
5       143000.0
6       307000.0
7       200000.0
8       129900.0
9       118000.0
10      129500.0
11      345000.0
12      144000.0
13      279500.0
14      157000.0
15      132000.0
16      149000.0
17       90000.0
18      159000.0
19      139000.0
20      325300.0
21      139400.0
22      230000.0
23      129900.0
24      154000.0
25      256300.0
26      134800.0
27      306000.0
28      207500.0
29       68500.0
30       40000.0
31      149350.0
32      179900.0
33      165500.0
34      277500.0
35      309000.0
36      145000.0
37      153000.0
38      109000.0
39       82000.0
40      160000.0
41      170000.0
42      144000.0
43      130250.0
44      141000.0
45      319900.0
46      239686.0
47      249700.0
48      113000.0
49      127000.0
50      177000.0
51      114500.0
52      110000.0
53      385000.0
54      130000.0
55      180500.0
56      172500.0
57      196500.0
58      438780

C:\Users\amallet\Anaconda\envs\udacity_env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [50]:
# Data preparation

class PrepareData(Dataset):

    def __init__(self, In, Out, scale_X=True):
        if not torch.is_tensor(In):
            if scale_X:
                #X = StandardScaler().fit_transform(X)
                In = In.to_numpy()
                self.In = torch.from_numpy(In)
        if not torch.is_tensor(Out):
            Out = Out.to_numpy()
            self.Out = torch.from_numpy(Out)

    def __len__(self):
        return len(self.In)

    def __getitem__(self, idx):
        return self.In[idx], self.Out[idx]

ds_train = PrepareData(train_data, label_train)
train_set = DataLoader(ds_train, batch_size=10)
ds_valid = PrepareData(valid_data, label_valid)
valid_set = DataLoader(ds_valid, batch_size=10)

image_dataset = {x: PrepareData(In=train_data if x == 'train'
                                else valid_data, 
                                Out=label_train if x == 'train'
                                else label_valid,
                                scale_X=True)
                for x in ['train', 'valid']}

data_loader = {x: torch.utils.data.DataLoader(image_dataset[x], batch_size = 10,  shuffle=True) 
               for x in ['train', 'valid']}

In [41]:
print(train_set)

In [42]:
for i, (data, target) in enumerate(train_set):
    print(data.shape)
    print(target.shape)
    next
    
    

torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([10, 1])
torch.Size([10, 288])
torch.Size([

'Model creation

In [157]:
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        
        # linear layer (330 -> 500)
        self.fc1 = nn.Linear(330, 500)
        
        # linear layer (500 -> 250)
        self.fc2 = nn.Linear(500, 250)
        
        # linear layer (250 -> 125)
        self.fc3 = nn.Linear(250, 125)
        
        # linear layer (125 -> 1)
        self.fc4 = nn.Linear(125, 1)
        
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.175)
        
        # LogSoftmax
        self.LSM = nn.LogSoftmax()
    
    def forward(self, x):
        
        # add 1st hidden layer, with relu activation function
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        #h2
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        #h3
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        #h4
        x = self.fc4(x)
        x = self.LSM(x)

        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_HR = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_patho.cuda()

In [158]:
model_HR

Net(
  (fc1): Linear(in_features=330, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=250, bias=True)
  (fc3): Linear(in_features=250, out_features=125, bias=True)
  (fc4): Linear(in_features=125, out_features=1, bias=True)
  (dropout): Dropout(p=0.175, inplace=False)
  (LSM): LogSoftmax()
)

In [121]:
### TODO: select loss function
criterion = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer = optim.SGD(model_HR.parameters(), lr=0.01, momentum = 0.9)

VERSION = 'Test_version'

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    time_start = time.time()
    train_class = []
    valid_class = []
    epoch_class = []
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        LR = 0.01
        
        ###################
        # train the model #
        ###################
        model.train()
        for idx, (data, target,_) in enumerate(loaders['train']):

            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            #optimizer.zero_grad()
            optim.SGD(model.parameters(), lr=LR, momentum=0.9).zero_grad()
            # add data.float() as the data is in ByteTensor format, which doesn't work the model data type
            output = model(data)
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            #optimizer.step()
            optim.SGD(model.parameters(), lr=LR, momentum=0.9).step()
            #update training loss
            train_loss += loss.item()*data.size(0)
            
        model.eval()
        for idx, (data, target, _) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            
        # calculate average losses
        train_loss = train_loss/len(loaders['train'].sampler)
        
        
        valid_loss = valid_loss/len(loaders['valid'].sampler)
        
        if valid_loss < 0.35 and valid_loss > 0.15:
            LR=0.005
        elif valid_loss < 0.15:
            LR=0.001
        
        
        # Calcul time
        time_now = time.time()
        
        time_epoch = (time_now - time_start)/60
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTime since the beginning {:.1f} min \tLearning rate: {:.6f} '.format(
            epoch, 
            train_loss,
            valid_loss,
            time_epoch,
            LR
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss,
            torch.save(model.state_dict(), VERSION))
                 )
            valid_loss_min = valid_loss
        
        # store class data
        train_class.append(train_loss)
        valid_class.append(valid_loss)
        epoch_class.append(epoch)
    
    plt.plot(epoch_class, train_class, 'g', label='Training loss')
    plt.plot(epoch_class, valid_class, 'b', label='validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    # return trained model
    return model